# `Занятие 9: Transformers + 🤗`
Воробьев Сергей | t.me/serv01 | sergei.vorobyov01@gmail.com

# `0. Вступление`

*Напиши краткое вступление к практическому занятию по трансформерам: почему это актуально и какие сейчас задачи решаются с помощью моделей архитектуры GPT*

> Добро пожаловать на наше практическое занятие по трансформерам! В последние годы модели на основе трансформеров, такие как BERT, GPT, и их производные, стали основным инструментом для решения качественных задач в области обработки естественного языка (NLP).

> Трансформеры представляют собой класс моделей, которые используют механизм внимания для генерации выходных данных по входным данным. Они стали основой для подавляющего большинства современных моделей обработки естественного языка, таких как GPT (Generative Pre-trained Transformer).

> Особенно сейчас, когда GPT-3 и его вариации определяют состояние техники в области генерации текста и обработки естественного языка, эти модели актуальны более чем когда-либо.

> С GPT вы можете решать широкий спектр задач, таких как генерация текста, перевод, ответы на вопросы, извлечение текста, семантическое представление и многие другие. Они значительно улучшили качество и разнообразие наших способностей по обработке языка и продолжают обладать огромным потенциалом.

> В течение этого занятия мы углубимся в изучение этих моделей, чтобы понять, как они устроены внутри, как их обучать, и как их использовать на практике в контексте различных задач NLP. Будет увлекательно, так что приготовьтесь познакомиться с миром трансформеров!

**На семинаре мы обсудим:**

1. Посмотрим откуда берутся тексты для обучения LLM и как они попадают в модель
2. Откуда берутся сами модели
3. Как они обучаются
4. Куда все это движется

# `1. Данные`

Для начала посмотрим на классический этап обучения ассистента на основе LLM:

<img src="https://miro.medium.com/v2/resize:fit:1200/1*WibZ0afyV8ZwaXjDx0Xu-g.png" width="700">

Автор: Andrej Karpathy, [лекция](https://www.youtube.com/watch?v=bZQun8Y4L2A)

In [1]:
%load_ext autoreload
%autoreload 2

In [38]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("./gemma-2b")
model = AutoModelForCausalLM.from_pretrained("./gemma-2b", device_map="mps")

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.
Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.26s/it]


In [41]:
# input_text = "Write reasons to become an ML engineer"
# input_text = "Россия это"
input_text = "Франция это Париж, Великобритания это Лондон, Россия это"
input_ids = tokenizer(input_text, return_tensors="pt").to("mps")

outputs = model.generate(**input_ids, max_length=50)
print(tokenizer.decode(outputs[0]))

<bos>Франция это Париж, Великобритания это Лондон, Россия это Москва, Китай это Пекин, Япония это Токио, США это Вашингтон, Австралия это Мельбурн, Индия это Нью-Дели


In [42]:
del model # иначе оперативка закончится
model_it = AutoModelForCausalLM.from_pretrained("./gemma-1.1-2b-it", device_map="mps")

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.63s/it]


In [43]:
input_text = "Write reasons to become an ML engineer"
# input_text = "Россия это"
# input_text = "Франция это Париж, Великобритания это Лондон, Россия это"
input_ids = tokenizer(input_text, return_tensors="pt").to("mps")

outputs = model_it.generate(**input_ids, max_length=50)
print(tokenizer.decode(outputs[0]))
del model_it

<bos>Write reasons to become an ML engineer.

**1. High Demand and Job Security:**

* The demand for ML engineers is expected to grow significantly in the coming years.
* Job security is high, with projections of high employment rates in


Подход к сборке и предобработке данных крайне важен при обучении больших языковых моделей. Этот этап влияет на качество и объективность модели, а также определяет, насколько хорошо модель будет справляться с реальными задачами.

- Для претрейна как правило данные собираются со всего интернета, они тщательно фильтруются и отбираются

- Далее в зависимости от задачи производится SFT, для ассистентов берут датасеты с инструкциями


Например, состав данных для претрейна может быть таким:

<img src="https://sun9-57.userapi.com/impg/_FV6r9zmUFv080cf9oGEYnc5mk5cnvC1kADYpg/9Nw1X7qU_kw.jpg?size=1602x1348&quality=95&sign=ad1911625ead2a02c0461dac503f0cc1&type=album" width="500">

https://arxiv.org/pdf/2302.13971.pdf

### Как собирают итоговый датасет претрейна

- Дедупликация
- Классификация и отбор
- Вычищение тестовых бенчмарков (Test set is all you need: https://arxiv.org/pdf/2309.08632.pdf)
- И так далее... (например AskLLM: https://arxiv.org/pdf/2402.09668.pdf)

### 🤗 Datasets

Датасет претрейна например: https://huggingface.co/datasets/allenai/c4

In [ ]:
# !pip3 install datasets

In [21]:
from datasets import load_dataset
instruct_dataset = load_dataset("WizardLM/WizardLM_evol_instruct_V2_196k")

Generating train split: 143000 examples [00:01, 82348.37 examples/s]


In [22]:
instruct_dataset["train"][2]

{'idx': 'zQCptr6',
 'conversations': [{'from': 'human',
   'value': 'Please develop a Python program that generates a list of prime numbers up to a specific upper limit value provided by the user by applying the Sieve of Eratosthenes algorithm, while ensuring that the program also optimizes the memory usage and runs in a timely manner even when dealing with extremely large datasets. Additionally, the program should have the ability to identify and exclude composite numbers from the list before outputting the final prime numbers.'},
  {'from': 'gpt',
   'value': 'Here is a Python program that uses the Sieve of Eratosthenes algorithm to generate a list of prime numbers up to a specific upper limit value provided by the user. It also optimizes the memory usage and runs in a timely manner even when dealing with extremely large datasets. Additionally, it identifies and excludes composite numbers from the list before outputting the final prime numbers:\r\n\r\n```python\r\nimport math\r\n\r\n

### Свой собственный датасет

In [44]:
with open("./wap1.txt", "rb") as f:
    part1 = f.read()
with open("./wap2.txt", "rb") as f:
    part2 = f.read()

In [45]:
war_and_peace = part1.decode("cp1251") + part2.decode("cp1251")
war_and_peace = war_and_peace.replace("\r", "")

In [46]:
len(war_and_peace)

3083048

In [47]:
war_and_peace[12312:13412]

' его отличали, взял за руку фрейлину, поцеловал ее и, поцеловав, помахал фрейлинскою рукой, развалившись на креслах и глядя в сторону.\n\n–\xa0Attendez,[27 - Постойте.] – сказала Анна Павловна, соображая.\xa0– Я нынче же поговорю с Lise (la femme du jeune Болконский).[28 - Лизе (жене Болконского).] И, может быть, это уладится. Ce sera dans votre famille que je ferai mon apprentissage de vieille fille.[29 - Я в вашем семействе начну обучаться ремеслу старой девицы.]\n\n\n\n\nII\n\n\nГостиная Анны Павловны начала понемногу наполняться. Приехала высшая знать Петербурга, люди самые разнородные по возрастам и характерам, но одинаковые по обществу, в каком все жили; приехала дочь князя Василия, красавица Элен, заехавшая за отцом, чтобы с ним вместе ехать на праздник посланника. Она была в шифре и бальном платье. Приехала и известная, как la femme la plus sеduisante de Pеtersbourg,[30 - самая обворожительная женщина в Петербурге.] молодая, маленькая княгиня Болконская, прошлую зиму вышедшая 

In [48]:
f = open("war_and_peace.txt", "a")
f.write(war_and_peace)
f.close()

# `2. Токенизация`

Нереально полезное видео чтобы погрузиться в токенизацию:

https://youtu.be/zduSFxRajkE?si=NVceMQolN5sTixAk

Очевидно, что мы не можем засунуть текст в модель – она ожидает получить числа.
В трансформерах используется обучаемая матрица ембеддингов, но нам все равно нужно перевести изначальное слово в OHE. И тут возникают проблемы:

1. В богатых на словоформы языках типа русского слов с одним корнем может быть очень много слов -> гигантская размерность словаря -> гигантская размерность обучаемой матрицы эмбеддингов.
2. Мы всегда встретим какое-то новое слово, являющееся формой или комбинацией уже известных, как например в немецком.

Эти проблемы помогает решить особый способ токенизации. Ниже разберем самый популярный вариант, который использовался в трансформере, а пока давайте посмотрим на иллюстрицию, которая помогает понять смысл токенизации:


<img src="https://www.oreilly.com/api/v2/epubs/9781492062561/files/assets/anlp_0401.png" width="700">

[источник картинки](https://www.oreilly.com/api/v2/epubs/9781492062561/files/assets/anlp_0401.png)

In [49]:
# !pip3 install sentencepiece # already in Google Colab
# https://github.com/google/sentencepiece
import sentencepiece as spm

In [ ]:
spm.SentencePieceTrainer.train(input="war_and_peace.txt", model_prefix='wap', vocab_size=1000)

In [50]:
tokenizer = spm.SentencePieceProcessor(model_file="./wap.model")

In [51]:
tokenizer.Encode("Привет, это Война и Мир")

[175, 111, 10, 68, 4, 106, 90, 69, 52, 9, 224, 8, 22]

In [52]:
tokenizer.Decode(tokenizer.Encode("Привет, это Война и Мир"))

'Привет, это Война и Мир'

Можно скачать токенизатор из Hugging Face

In [53]:
from transformers import PreTrainedTokenizerFast
tokenizer1 = PreTrainedTokenizerFast.from_pretrained("gpt2")
tokenizer1.add_special_tokens({'pad_token': '[PAD]'})

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


1

In [54]:
tokenizer1.encode("Привет, это война и мир")

[140,
 253,
 21169,
 18849,
 38857,
 16843,
 20375,
 11,
 220,
 141,
 235,
 20375,
 15166,
 12466,
 110,
 25443,
 117,
 22177,
 16142,
 12466,
 116,
 12466,
 120,
 18849,
 21169]

In [60]:
tokenizer1.decode([11])

','

Песочница: https://tiktokenizer.vercel.app/

Что бывает когда вы обучаете токенизатор на данных, которых нет в претрейне:

https://www.lesswrong.com/posts/aPeJE8bSo6rAFoLqg/solidgoldmagikarp-plus-prompt-generation

Создадим датасет:

In [61]:
import torch
from torch.utils.data import Dataset, DataLoader

class WarAndPeaceDataset(Dataset):
    def __init__(self, rawtext, tokenizer, max_seq_len=32):
        # lines = rawtext.split('\n')
        self.tokenized_dataset = tokenizer.encode(rawtext)
        self.max_seq_len = 32

    def __len__(self):
        return len(self.tokenized_dataset)

    def __getitem__(self, i):
        idx = torch.randint(0, len(self.tokenized_dataset) - self.max_seq_len - 1, (1,))
        seq = torch.Tensor(self.tokenized_dataset[idx.item():idx.item()+self.max_seq_len+1]).to(torch.long)
        return seq[:-1], seq[1:]

# `3. Создание transformer-based модели`

Не хочется писать велосипед, лучше потратить время с пользой и полистать чей-нибудь код

In [ ]:
!git clone https://github.com/karpathy/minGPT.git
!pip3 install -e minGPT

In [22]:
# import sys
# sys.path.append("/Users/serv01/Documents/gitprojects/2024_seminar/minGPT")

In [62]:
from mingpt.model import GPT
model_config = GPT.get_default_config()
model_config.model_type = 'gpt2'
model_config.vocab_size = 1000 # openai's model vocabulary
model_config.block_size = 32  # openai's model block_size (i.e. input context length)
model = GPT(model_config)

number of parameters: 85.85M


In [65]:
train_dataset = WarAndPeaceDataset(war_and_peace, tokenizer)

from mingpt.trainer import Trainer
train_config = Trainer.get_default_config()
train_config.learning_rate = 5e-4 # many possible options, see the file
train_config.max_iters = 1000
train_config.batch_size = 32
train_config.device = "cpu"
trainer = Trainer(train_config, model, train_dataset)

running on device cpu


# `4. Обучение transformer-based модели`

Запускаем обучение:

In [ ]:
trainer.run()

Обязательно сохраняем чекпоинт:

In [ ]:
# torch.save(model.state_dict(), "86M_wap_m1.pt")

Обучать модель лучше с помощью [готовых инструментов](https://huggingface.co/docs/transformers/v4.39.3/en/main_classes/trainer):

```python
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)
```

Лучше сохранять по ходу обучения: https://pytorch.org/tutorials/recipes/recipes/saving_and_loading_a_general_checkpoint.html

In [67]:
model_state_dict = torch.load("86M_wap.pt", map_location="cpu")
tokenizer = spm.SentencePieceProcessor(model_file="./true_wap.model") # это тоже очень важно!!!

In [73]:
model_config = GPT.get_default_config()
model_config.model_type = 'gpt2'
model_config.vocab_size = 1000 # openai's model vocabulary
model_config.block_size = 32  # openai's model block_size (i.e. input context length)
model1 = GPT(model_config)
model1.load_state_dict(model_state_dict)

number of parameters: 85.85M


<All keys matched successfully>

In [74]:
model1.to("mps")
batch = torch.Tensor([tokenizer.Encode("Он раскрыл глаза, надеясь увидать, чем кончилась борьба французов")]).to(torch.long).to("mps")
output = model1.generate(batch, max_new_tokens=200)
tokenizer.Decode(output.tolist()[0])

'Он раскрыл глаза, надеясь увидать, чем кончилась борьба французов. –  ⁇ ui, il faut ? l’Empereur, il faut ? le prince, – сказал он, – que j’ai prieux, – сказал Шиншин, – – il caus pour fait pour tout pour moi,[337 - Государь, – сказал Шиншин, – а то, что ей было то, что он не мог отстать от них, но не мог не отстать от них. –  ⁇ icolas, – сказал он, – c’est bel fait pour le pri'

### Хитрости обучения

> К сожалению, если вы просто напишете код Transformer-нейросети и попробуете сразу обучить что-то содержательное, используя привычные для других архитектур гиперпараметры, то вас с большой вероятностью постигнет неудача. Оптимизационный процесс для таких моделей зачастую требуется изменить, и недостаточное внимание к этому может повлечь за собой существенные потери в итоговом качестве или вообще привести к нестабильному обучению.

> Первый момент, на который стоит обратить внимание, — размер батча для обучения. Практически все современные Transformer-модели обучаются на больших батчах, которые для самых больших языковых моделей могут достигать миллионов токенов. Разумеется, ни одна современная GPU не может обработать столько данных за один шаг: на помощь приходят распределенное обучение и чуть более универсальный трюк с аккумуляцией градиентов по микробатчам. Также в последних статьях зачастую прибегают к увеличению размера батча по ходу обучения: идея заключается в том, что на ранних этапах важнее быстрее совершить много шагов градиентного спуска, а на поздних становится важнее иметь точную оценку градиента.

Я кратко пройдусь по некоторым опциям, которые могут быть полезны:

1. Optimizers
2. Warm up + LR schedulers
2. Mixed precision training
3. Gradient accumulating

**Optimizers**

Вот так выглядит Adam

<img src="https://sun9-80.userapi.com/impg/x7er3KBsSCCHYtaEcOU72hm0Dg-GoRBh99wOqQ/LmwnU6U_ufY.jpg?size=858x374&quality=95&sign=05f1ce2f2a52fe7e62a9902fd92809b4&type=album" width=450>

> Adam требует хранения как параметров модели, так и градиентов, накопленного импульса и нормировочных констант (cache). Т.е. достижение более быстрой (с точки зрения количества итераций/объема рассмотренных данных) сходимости требует больших объемов памяти. Кроме того, если вы решите продолжить обучение модели, остановленное на некоторой точке, необходимо восстановить из чекпоинта не только веса модели, но и накопленные параметры Adam. В противном случае оптимизатор начнёт сбор всех своих статистик с нуля, что может сильно сказаться на качестве дообучения. То же самое касается вообще всех описанных выше методов, так как каждый из них накапливает какие-то статистики во время обучения.

Источник: https://education.yandex.ru/handbook/ml/article/optimizaciya-v-ml

Оптимизаторы занимают очень много места

<img src="https://sun9-38.userapi.com/impg/mUJ4sI7yrYAwpJo2IIIYwt1912vxip50bV-1Lg/YPJDDNYpTBQ.jpg?size=2306x1370&quality=95&sign=8f95b4865127bf44782af2b684a1673b&type=album" width=600>

Статья про ZeRO: https://arxiv.org/pdf/1910.02054.pdf

**Warm up + LR Scheduler**

В [оригинальной статье](https://arxiv.org/pdf/1706.03762.pdf) про трансформер:

<img src="https://sun9-8.userapi.com/impg/NIeh1dxKgnWtvg9T6U55_VDzqxwTMWfbv5uYjQ/daqoujlQb2I.jpg?size=1592x726&quality=95&sign=9fee44549dc43428df481d6d1c4497fa&type=album" width="700">

Этот скорее эвристичесикй метод используется для изменения learning rate во время обучения, добавляется просто параметрами `Trainer`

Гайд на Kaggle: https://www.kaggle.com/code/isbhargav/guide-to-pytorch-learning-rate-scheduling

**Mixed precision**

Наша модель обучается с помощью float32 градиентов. Возможно, эта точность излишняя, давайте попробуем обучать используя float16. Очень часто, этой точночти хватает для тренировки, при этом мы получаем существенное ускорение процесса обучения:

In [ ]:
additional_training_args = {
    "fp16": True,
}

<img src="https://developer-blogs.nvidia.com/wp-content/uploads/2019/01/pasted-image-0-21.png" width=600>

Ссылки:
- [Blog post about reduced precision FP formats](https://moocaholic.medium.com/fp64-fp32-fp16-bfloat16-tf32-and-other-members-of-the-zoo-a1ca7897d407)
- [Blog posts about mixed precision training with Tensor Cores](https://developer.nvidia.com/blog/video-mixed-precision-techniques-tensor-cores-deep-learning/)
- [AMP and other Torch magic](https://pytorch.org/tutorials/recipes/recipes/tuning_guide.html#)

Некоторые кстати пошли еще дальше...

<img src="https://sun9-8.userapi.com/impg/xs3qYhaXa2gdmaaN_JUqyA0AuQu7urGU4kSePA/JczrKQ_6Wbg.jpg?size=2390x782&quality=95&sign=06130557254fc140922eaba79b198469&type=album" width=900>

<img src="https://docs.nvidia.com/deeplearning/transformer-engine/user-guide/_images/fp8_formats.png" width=600>

https://arxiv.org/abs/2310.18313

**Gradient accumulating**

Часто, желаемый размер батча просто не помещается в нашу видеокарту. Давайте просто делить батч на несколько подбатчей и аккумульровать результаты:

Tutorial: https://huggingface.co/docs/accelerate/en/usage_guides/gradient_accumulation

In [ ]:
additional_training_args = {
    'per_device_train_batch_size': 12,
    'per_device_eval_batch_size': 12,
    'gradient_accumulation_steps': 2,
    'gradient_checkpointing': True,
}

<img src="https://substackcdn.com/image/fetch/f_auto,q_auto:good,fl_progressive:steep/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2F6f60f972-12e9-4b6a-908e-a287a0273605_3640x2216.png" width=500>

Source: https://blog.dailydoseofds.com/p/gradient-accumulation-increase-batch

# `5. Дообучение готовых моделей, LoRA`

Еще одним подходом, который ускоряет fine-tunning, является Low-Rank Adaptation

<img src="https://adapterhub.ml/static/images/lora.png" width="400">

Low-Rank Adaptation (LoRA) — это эффективный метод дообучения, предложенный [Hu et al. (2021)](https://arxiv.org/pdf/2106.09685.pdf). LoRA вводит обучаемые матрицы разложения низкого ранга в слои предварительно обученной модели. Поэтому для любого слоя модели, выраженного в виде умножения матриц формы $h=W_0x$, он выполняет перепараметризацию, так что:
$$
h = W_0x + \frac{\alpha}{r}BAx
$$
где $A \in R^{r \times k}$ $B \in R^{d \times r}$ – являются матрицами разложения, а $r$ – низкоразмерный ранг разложения, является наиболее важным гиперпараметром.

Хотя, в принципе, эта репараметризация может быть применена к любой матрице весов в модели, исходная статья адаптирует только веса self-attention. `adapter-transformers` дополнительно позволяют внедрять LoRA в FCN слои в промежуточных и выходных компонентах блока Transformer. Вы можете настроить места, в которые должны быть добавлены веса LoRA, используя атрибуты в классе `LoRAConfig`.

https://huggingface.co/docs/diffusers/en/training/lora

In [ ]:
from transformers.adapters import LoRAConfig

config = LoRAConfig(r=8, alpha=16)
model.add_adapter("lora_adapter", config=config)

В статье [Hu et al. (2021)](https://arxiv.org/pdf/2106.09685.pdf) также уделяют особое внимание тому, чтобы свести к минимуму время обучения по сравнению с полным дообучением. Для этого репараметризация LoRA может быть объединена с исходными предварительно обученными весами модели. Таким образом, веса адаптера напрямую используются в каждом forward pass без передачи активаций через дополнительный модуль. В `adapter-transformers`
 это можно реализовать с помощью встроенного метода `merge_adapter()`:

In [ ]:
model.merge_adapter("lora_adapter")

Чтобы продолжить обучение с этим адаптером LoRA или полностью отключить его, сначала необходимо снова сбросить объединенные веса:

In [ ]:
model.reset_adapter("lora_adapter")

**Материалы:**

1. https://arxiv.org/pdf/2106.09685.pdf
2. https://adapterhub.ml/blog/2022/09/updates-in-adapter-transformers-v3-1/

# `6. Дальнейшее развитие трансформеров`

LLaMA

<img src="https://sun9-4.userapi.com/impg/c3yM2t5AfMDMf0_c5CL0efV5fgm0JUFPhhUIrQ/sGWCwmdMGA4.jpg?size=1400x789&quality=95&sign=252ca0155b08bc60ccc846b012aff019&type=album" width=700>

MoE

<img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/blog/moe/00_switch_transformer.png" width=600>

[MoE Explained](https://huggingface.co/blog/moe)

Следить за топом моделей можно тут:

[LMSYS Chatbot Arena Leaderboard](https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard)

# `(Opt.) 6.1 Embeddings`

> The Embeddings class is a class designed for interfacing with text embedding models. There are lots of embedding model providers (OpenAI, Cohere, Hugging Face, etc) - this class is designed to provide a standard interface for all of them.

> Embeddings create a vector representation of a piece of text. This is useful because it means we can think about text in the vector space, and do things like semantic search where we look for pieces of text that are most similar in the vector space.

> The base Embeddings class in LangChain provides two methods: one for embedding documents and one for embedding a query. The former takes as input multiple texts, while the latter takes a single text. The reason for having these as two separate methods is that some embedding providers have different embedding methods for documents (to be searched over) vs queries (the search query itself).

- Source: https://developers.sber.ru/docs/ru/gigachat/sdk/modules/data-connection/text-embedding/overview
- Nice paper: https://arxiv.org/pdf/2401.00368.pdf
- Nice model: https://huggingface.co/intfloat/e5-mistral-7b-instruct

# `7. Доп материалы`

- [ML handbook](https://education.yandex.ru/handbook/ml/article/transformery)
- [Let's build GPT: from scratch, in code, spelled out.](https://www.youtube.com/watch?v=kCc8FmEb1nY)
- [Scaling ChatGPT: Five Real-World Engineering Challenges](https://newsletter.pragmaticengineer.com/p/scaling-chatgpt)
- [Current Best Practices for Training LLMs from Scratch](https://wandb.ai/site/resources/whitepapers/llm-whitepaper)